### **5. Modeling**
---

We will perform:
1. Forward Selection Procedure
2. Best Model Adjustment

#### **Perform Forward Selection Procedure**
---
Begin with null model (no predictors), then adds predictor that gives the greatest additional improvement to the model, one-at-a-time.

In [1]:
# Import library
import pandas as pd
import numpy as np

# Import library for modeling
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression

# Load configuration
import src.utils as utils

In [2]:
CONFIG_DATA = utils.config_load()
CONFIG_DATA

{'raw_dataset_path': 'data/raw/credit_risk_dataset.csv',
 'dataset_path': 'data/output/data.pkl',
 'predictors_set_path': 'data/output/predictors.pkl',
 'response_set_path': 'data/output/response.pkl',
 'train_path': ['data/output/X_train.pkl', 'data/output/y_train.pkl'],
 'test_path': ['data/output/X_test.pkl', 'data/output/y_test.pkl'],
 'data_train_path': 'data/output/data_train.pkl',
 'data_train_binned_path': 'data/output/data_train_binned.pkl',
 'crosstab_list_path': 'data/output/crosstab_list.pkl',
 'WOE_table_path': 'data/output/WOE_table.pkl',
 'IV_table_path': 'data/output/IV_table.pkl',
 'WOE_map_dict_path': 'data/output/WOE_map_dict.pkl',
 'X_train_woe_path': 'data/output/X_train_woe.pkl',
 'response_variable': 'loan_status',
 'test_size': 0.3,
 'num_columns': ['person_age',
  'person_income',
  'person_emp_length',
  'loan_amnt',
  'loan_int_rate',
  'loan_percent_income',
  'cb_person_cred_hist_length'],
 'cat_columns': ['person_home_ownership',
  'loan_intent',
  'loan_g

Define function `forward()` to fit a model on the train set and calculate its CV score from the validation set.

In [3]:
# Function to perform forward selection procedure
def forward(X, y, predictors, scoring='roc_auc', cv=5):
    """Function to perform forward selection procedure"""

    # Define sample size and  number of all predictors
    n_samples, n_predictors = X.shape

    # Define list of all predictors
    col_list = np.arange(n_predictors)

    # Define remaining predictors for each k
    remaining_predictors = [p for p in col_list if p not in predictors]

    # Initialize list of predictors and its CV Score
    pred_list = []
    score_list = []

    # Cross validate each possible combination of remaining predictors
    for p in remaining_predictors:
        combi = predictors + [p]

        # Extract predictors combination
        X_ = X[:, combi]
        y_ = y

        # Define the estimator
        model = LogisticRegression(penalty = None,
                                   class_weight = 'balanced')

        # Cross validate the recall scores of the model
        cv_results = cross_validate(estimator = model,
                                    X = X_,
                                    y = y_,
                                    scoring = scoring,
                                    cv = cv)

        # Calculate the average CV/recall score
        score_ = np.mean(cv_results['test_score'])

        # Append predictors combination and its CV Score to the list
        pred_list.append(list(combi))
        score_list.append(score_)

    # Tabulate the results
    models = pd.DataFrame({"Predictors": pred_list,
                           "CV Score": score_list})

    # Choose the best model
    best_model = models.loc[models['CV Score'].argmax()]

    return models, best_model

In [4]:
# Function to perform forward selection on all characteristics
def run_forward():
    """Function to perform forward selection on all characteristics"""

    cv = CONFIG_DATA['num_of_cv']
    scoring = CONFIG_DATA['scoring']

    X_train_woe_path = CONFIG_DATA['X_train_woe_path']
    X_train_woe = utils.pickle_load(X_train_woe_path)
    X_train = X_train_woe.to_numpy()

    y_train_path = CONFIG_DATA['train_path'][1]
    y_train = utils.pickle_load(y_train_path)
    y_train = y_train.to_numpy()

    # First, fit the null model
    # Define predictor for the null model
    predictor = []

    # The predictor in the null model is zero values for all predictors
    X_null = np.zeros((X_train.shape[0], 1))

    # Define the estimator
    model = LogisticRegression(penalty = None,
                               class_weight = 'balanced')

    # Cross validate
    cv_results = cross_validate(estimator = model,
                                X = X_null,
                                y = y_train,
                                cv = cv,
                                scoring = scoring)

    # Calculate the average CV score
    score_ = np.mean(cv_results['test_score'])

    # Create table for the best model of each k predictors
    # Append the results of null model
    forward_models = pd.DataFrame({"Predictors": [predictor],
                                   "CV Score": [score_]})

    # Next, perform forward selection for all predictors
    # Define list of predictors
    predictors = []
    n_predictors = X_train.shape[1]

    # Perform forward selection procedure for k=1,...,n_predictors
    for k in range(n_predictors):
        _, best_model = forward(X = X_train,
                                y = y_train,
                                predictors = predictors,
                                scoring = scoring,
                                cv = cv)

        # Tabulate the best model of each k predictors
        forward_models.loc[k+1] = best_model
        predictors = best_model['Predictors']

    # Find the best CV score
    best_idx = forward_models['CV Score'].argmax()
    best_cv_score = forward_models['CV Score'].loc[best_idx]
    best_predictors = forward_models['Predictors'].loc[best_idx]

    # Print the summary
    print('===================================================')
    print('Best index            :', best_idx)
    print('Best CV Score         :', best_cv_score)
    print('Best predictors (idx) :', best_predictors)
    print('Best predictors       :')
    print(X_train_woe.columns[best_predictors].tolist())
    print('===================================================')

    print(forward_models)
    print('===================================================')
    
    forward_models_path = CONFIG_DATA['forward_models_path']
    utils.pickle_dump(forward_models, forward_models_path)

    best_predictors_path = CONFIG_DATA['best_predictors_path']
    utils.pickle_dump(best_predictors, best_predictors_path)

    return forward_models, best_predictors

In [5]:
# Run the function
run_forward()

FileNotFoundError: [Errno 2] No such file or directory: 'data/output/X_train_woe.pkl'

In [14]:
# Function to fit the best model on whole X_train
def best_model_fitting(best_predictors):
    """Function to fit best model on whole X_train"""

    X_train_path = CONFIG_DATA['X_train_woe_path']
    X_train_woe = utils.pickle_load(X_train_path)
    X_train = X_train_woe.to_numpy()

    y_train_path = CONFIG_DATA['train_path'][1]
    y_train = utils.pickle_load(y_train_path)
    y_train = y_train.to_numpy()

    if best_predictors is None:
        best_predictors_path = CONFIG_DATA['best_predictors_path']
        best_predictors = utils.pickle_load(best_predictors_path)
        print(f"Best predictors index   :", best_predictors)
    else:
        print(f"[Adjusted] best predictors index   :", best_predictors)

    # Define X with best predictors
    X_train_best = X_train[:, best_predictors]

    # Fit best model
    best_model = LogisticRegression(penalty = None,
                                    class_weight = 'balanced')
    best_model.fit(X_train_best, y_train)

    print(best_model)

    # Extract the best model' parameter estimates
    best_model_intercept = pd.DataFrame({'Characteristic': 'Intercept',
                                         'Estimate': best_model.intercept_})
    
    best_model_params = X_train_woe.columns[best_predictors].tolist()

    best_model_coefs = pd.DataFrame({'Characteristic': best_model_params,
                                     'Estimate': np.reshape(best_model.coef_, 
                                                            len(best_predictors))})

    best_model_summary = pd.concat((best_model_intercept, best_model_coefs),
                                   axis = 0,
                                   ignore_index = True)
    
    print('===================================================')
    print(best_model_summary)
    
    best_model_path = CONFIG_DATA['best_model_path']
    utils.pickle_dump(best_model, best_model_path)

    best_model_summary_path = CONFIG_DATA['best_model_summary_path']
    utils.pickle_dump(best_model_summary, best_model_summary_path)

    return best_model, best_model_summary

In [15]:
# Check the function
best_model_fitting(best_predictors = None)

Best predictors index   : [2, 9]
LogisticRegression(class_weight='balanced', penalty=None)
              Characteristic  Estimate
0                  Intercept  0.000216
1      person_home_ownership -0.996248
2  cb_person_default_on_file -0.991864


(LogisticRegression(class_weight='balanced', penalty=None),
               Characteristic  Estimate
 0                  Intercept  0.000216
 1      person_home_ownership -0.996248
 2  cb_person_default_on_file -0.991864)

#### **Best Model Adjustment**
---

Scorecards with too few characteristics are generally unable to withstand the test of time:
  - They are susceptible to minor changes in the applicant profile.
  - A good adjudicator will never look at just two characteristics from an application form to make a decision.

We will include all characteristics in the final model.
  - From the independence test, all characteristics are not independent of the response variable (probability of default).
  - Generally, a final scorecards consist of between 8 and 15 characteristics

In [16]:
# Adjust the best predictors
best_model_fitting(best_predictors = np.arange(11).tolist())

[Adjusted] best predictors index   : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


LogisticRegression(class_weight='balanced', penalty=None)
                Characteristic  Estimate
0                    Intercept -0.059463
1                   person_age -0.057122
2                person_income -0.977773
3        person_home_ownership -0.749994
4            person_emp_length -0.249803
5                  loan_intent -1.252916
6                   loan_grade -1.156946
7                    loan_amnt -0.855171
8                loan_int_rate  0.008703
9          loan_percent_income -0.765774
10   cb_person_default_on_file  0.017819
11  cb_person_cred_hist_length -0.569405


(LogisticRegression(class_weight='balanced', penalty=None),
                 Characteristic  Estimate
 0                    Intercept -0.059463
 1                   person_age -0.057122
 2                person_income -0.977773
 3        person_home_ownership -0.749994
 4            person_emp_length -0.249803
 5                  loan_intent -1.252916
 6                   loan_grade -1.156946
 7                    loan_amnt -0.855171
 8                loan_int_rate  0.008703
 9          loan_percent_income -0.765774
 10   cb_person_default_on_file  0.017819
 11  cb_person_cred_hist_length -0.569405)